# Tarea - Clasificacion Binaria de Pinturas para Auto

In [2]:
import random
import numpy as np
import pandas as pd
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from optbinning import BinningProcess
from optbinning import OptimalBinning

load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\pablo\anaconda3\envs\master_ucm_machine_learning\Lib\site-packages\ortools\.libs\ortools.dll...
(CVXPY) Apr 17 04:25:20 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.12.4544). Expected < 9.12.0. Please open a feature request on cvxpy to enable support fo

## 1 - Importación y Depuración de Datos

Importamos el conjunto de datos y lo depuramos aplicando 3 transformaciones al mismo

1. Tipos de datos: asignamos los tipos de datos a cada variables. A Airbags la vamos a tratar como categóricas por tener saltos no continuos para que pueda capturar no linealidades posibles a futuro.

2. Datos missings: imputamos a los datos missings de variables categóricas la moda y a los de variables numéricas la media

3. Estandarización: estandarizamos las variables tratadas como continuas usando la técnica a escala entre 0 y 1.

4. Dummies en Categóricas: agregamos columnas dummys para los n-1 valores posibles de cada variable categórica

In [13]:
bronze_data = pd.read_excel('../Statement/datos_tarea25.xlsx')

silver_data = bronze_data.copy()  # Crear una copia del dataset original


In [14]:
silver_data['Color'] = silver_data['Color'].map({'White': 1, 'Black': 0})

In [15]:
warnings.filterwarnings("ignore", category=FutureWarning)

# Reemplazar "-" por NaN y convertir a numérico
silver_data['Levy'] = silver_data['Levy'].replace("-", np.nan)
silver_data['Levy'] = pd.to_numeric(silver_data['Levy'], errors='coerce')

# Excluir valores nulos de Levy para el binning
non_null_data = silver_data[silver_data['Levy'].notnull()]

# Configurar el proceso de binning con el criterio de Information Value (IV)
binning_process = BinningProcess(
    variable_names=['Levy'],
    selection_criteria={"iv": {"min": 0.02}},
    max_n_bins=5  # Máximo número de bins permitido
)

# Ajustar el binning process a los datos no nulos
binning_process.fit(non_null_data[['Levy']], non_null_data['Color'])

# Obtener los puntos de corte (splits)
levy_binned = binning_process.get_binned_variable('Levy')
binning_table = levy_binned.binning_table
binning_table.build()
levy_splits = binning_table.splits

# Crear etiquetas personalizadas para los tramos
levy_bins_labels = [f"{float(lower)} - {float(upper)}" for lower, upper in zip([-float('inf')] + list(levy_splits), list(levy_splits) + [float('inf')])]

# Asignar las etiquetas personalizadas a los tramos
silver_data['Levy_bins'] = pd.cut(
    silver_data['Levy'],
    bins=[-float('inf')] + list(levy_splits) + [float('inf')],
    labels=levy_bins_labels,
    right=False
)

# Añadir un tramo específico para los valores nulos
silver_data['Levy_bins'] = silver_data['Levy_bins'].cat.add_categories("Missing")
silver_data['Levy_bins'].fillna("Missing", inplace=True)
silver_data = silver_data.drop(columns=['Levy'])

In [16]:
# Convertir las columnas a sus tipos de datos correctos
silver_data['Mileage'] = silver_data['Mileage'].str.replace(' km', '').str.replace(',', '').astype(float)  # Convertir Mileage a numérico
silver_data['Engine volume'] = silver_data['Engine volume'].str.replace(' Turbo', '').astype(float)  # Convertir Engine volume a numérico
silver_data['Prod. year'] = silver_data['Prod. year'].astype(int)  # Asegurar que Prod. year sea entero


In [17]:
silver_data['Prod. year'] = pd.to_numeric(silver_data['Prod. year'], errors='coerce')
non_null_data = silver_data[silver_data['Prod. year'].notnull()]

# Crear tramos manuales para 'Prod. year'
manual_bins = [1943, 2000, 2004, 2008, 2012, 2016, 2020]
manual_labels = ['1943-2000', '2001-2004', '2005-2008', '2009-2012', '2013-2016', '2017-2020']

# Asignar los tramos manuales a la columna 'Prod. year'
silver_data['Prod. year_bins'] = pd.cut(
    silver_data['Prod. year'],
    bins=manual_bins,
    labels=manual_labels,
    right=True
)

In [18]:
for column in silver_data.columns:
    if silver_data[column].dtype == 'object':  # Imputar moda para variables categóricas
        silver_data[column].fillna(silver_data[column].mode()[0], inplace=True)
    elif silver_data[column].dtype in ['int64', 'float64']:  # Imputar promedio para variables numéricas
        silver_data[column].fillna(silver_data[column].mean(), inplace=True)

In [20]:
# 2. Crear columnas dummys para variables categóricas

silver_data = pd.get_dummies(silver_data, columns=silver_data.drop(columns=['Color']).select_dtypes(include=['object']).columns, drop_first=True)

In [21]:
# 3. Estandarizar variables continuas a escala [0, 1]
scaler = MinMaxScaler()
numerical_columns = silver_data.select_dtypes(include=['int64', 'float64']).columns
silver_data[numerical_columns] = scaler.fit_transform(silver_data[numerical_columns])

Dividimos los conjuntos de datos para train y test usando una semilla de aleatoriedad prefijada

In [22]:
Y = silver_data['Color'] 
X = silver_data.drop(columns=['Color'])
print("Variables explicativas: ", list(X.columns))
print("Variable objetivo: ", Y.name)

SEED = 12345
random.seed(SEED)
np.random.seed(SEED)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=SEED)

Variables explicativas:  ['Price', 'Levy', 'Prod. year', 'Engine volume', 'Mileage', 'Cylinders', 'Airbags', 'Manufacturer_LEXUS', 'Manufacturer_MERCEDES-BENZ', 'Manufacturer_TOYOTA', 'Category_Jeep', 'Category_Sedan', 'Leather interior_Yes', 'Fuel type_Hybrid', 'Fuel type_Petrol', 'Gear box type_Tiptronic', 'Drive wheels_Front', 'Drive wheels_Rear', 'Wheel_Right-hand drive']
Variable objetivo:  Color


## 2 - Modelización con maquina de vector soporte